In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

In [3]:
def create_assistant(name,instruction,model):
    assistant=client.beta.assistants.create(
    name=name,
    instructions=instruction,
    model=model
    )
    return assistant

In [4]:
def create_thread():
    thread = client.beta.threads.create()
    return thread

In [5]:
def create_message(thread_id,role,content):
    message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role=role,
    content=content
)

In [6]:
def run_assistant(thread,assistant):
    run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
    )

    messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

    message_content = messages[0].content[0].text
    annotations = message_content.annotations
    citations = []
    for index, annotation in enumerate(annotations):
        message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
        if file_citation := getattr(annotation, "file_citation", None):
            cited_file = client.files.retrieve(file_citation.file_id)
            citations.append(f"[{index}] {cited_file.filename}")

    print(message_content.value)
    print("\n".join(citations))

In [7]:
#Initialization

assistant=create_assistant("Intelligent Document Processing Assistant","You are intellignet document processing assistant, fetch the data from the documents and analyse it.","gpt-3.5-turbo")
thread=create_thread()
create_message(thread.id,"user","""Use the schema to extract the data from the context and analyse it.
json_schema:\r\n\r\n\r\n{\n  \"title\": \"PlanList\",\n  \"definitions\": {\n    \"EnrollmentActual\": {\n      \"title\": \"EnrollmentActual\",\n      \"type\": \"object\",\n      \"properties\": {\n        \"EmployeeOnly\": {\n          \"title\": \"Employee Only\",\n          \"description\": \"Enrolled Actual employee count\",\n          \"type\": \"string\"\n        },\n        \"EmployeeSpouse\": {\n          \"title\": \"Employee Spouse\",\n          \"description\": \"Enrolled Actual Employee spouse count\",\n          \"type\": \"string\"\n        },\n        \"EmployeeChild\": {\n          \"title\": \"Employee Child\",\n          \"description\": \"Enrolled Actual Employee Child count\",\n          \"type\": \"string\"\n        },\n        \"Family\": {\n          \"title\": \"Family\",\n          \"description\": \"Enrolled Actual Employee Family count\",\n          \"type\": \"string\"\n        }\n      },\n      \"required\": [\n        \"EmployeeOnly\",\n        \"EmployeeSpouse\",\n        \"EmployeeChild\",\n        \"Family\"\n      ]\n    },\n    \"InNetworkDetails\": {\n      \"title\": \"InNetworkDetails\",\n      \"type\": \"object\",\n      \"properties\": {\n        \"DeductibleIndividual\": {\n          \"title\": \"Deductible Individual\",\n          \"type\": \"string\"\n        },\n        \"DeductibleFamily\": {\n          \"title\": \"Deductible Family\",\n          \"type\": \"string\"\n        },\n        \"OutofPocketFamily\": {\n          \"title\": \"Out of Pocket Family\",\n          \"type\": \"string\"\n        },\n        \"OutofPocketIndividual\": {\n          \"title\": \"Out of Pocket Individual\",\n          \"type\": \"string\"\n        },\n        \"OfficeVisit\": {\n          \"title\": \"Office Visit\",\n          \"description\": \"map only PCP value\",\n          \"type\": \"string\"\n        },\n        \"Coinsurance\": {\n          \"title\": \"Coinsurance\",\n          \"type\": \"string\"\n        },\n        \"InpatientHospitalServices\": {\n          \"title\": \"Inpatient Hospital Services\",\n          \"type\": \"string\"\n        },\n        \"OutpatientSurgery\": {\n          \"title\": \"Outpatient Surgery\",\n          \"type\": \"string\"\n        },\n        \"SpecialtyOfficeVisit\": {\n          \"title\": \"Specialty Office Visit\",\n          \"description\": \"map only SPC value\",\n          \"type\": \"string\"\n        },\n        \"UrgentCare\": {\n          \"title\": \"Urgent Care\",\n          \"type\": \"string\"\n        },\n        \"EmergencyRoom\": {\n          \"title\": \"Emergency Room\",\n          \"type\": \"string\"\n        },\n        \"Lab\": {\n          \"title\": \"Lab\",\n          \"description\": \"You can take value from 'Lab Services - Ind. Lab' field, must match this type in percentage\",\n          \"type\": \"string\"\n        },\n        \"XRay\": {\n          \"title\": \"Equivalent X-Ray\",\n          \"description\": \"Is equivalent value of 'Lab Services - Ind. Lab, usually in percentage value\",\n          \"type\": \"string\"\n        },\n        \"AdvancedImaging\": {\n          \"title\": \"Advanced Imaging\",\n          \"description\": \"Is equivalent to 'Adv. Radiology - Outpatient'\",\n          \"type\": \"string\"\n        },\n        \"RXTiers\": {\n          \"title\": \"RX Tiers\",\n          \"description\": \"RX Tiers value mapped across Retails which falls under pharmacy\",\n          \"type\": \"string\"\n        },\n        \"RXDeductible\": {\n          \"title\": \"RX Deductible\",\n          \"description\": \"Rx-Deductible field if the current plan name has 'HSA' or 'HSA Compatible' assign the string 'Plan Deductible' else if the current plan name is 'PPO' or 'OAP' mark it as NA\",\n          \"type\": \"string\"\n        }\n      },\n      \"required\": [\n        \"DeductibleIndividual\",\n        \"DeductibleFamily\",\n        \"OutofPocketFamily\",\n        \"OutofPocketIndividual\",\n        \"OfficeVisit\",\n        \"Coinsurance\",\n        \"InpatientHospitalServices\",\n        \"OutpatientSurgery\",\n        \"SpecialtyOfficeVisit\",\n        \"UrgentCare\",\n        \"EmergencyRoom\",\n        \"Lab\",\n        \"XRay\",\n        \"AdvancedImaging\",\n        \"RXTiers\",\n        \"RXDeductible\"\n      ]\n    },\n    \"LevelFundedInformation\": {\n      \"title\": \"LevelFundedInformation\",\n      \"type\": \"object\",\n      \"properties\": {\n        \"IndividualStopLoss\": {\n          \"title\": \"Individual Stop Loss\",\n          \"description\": \"Must be in USD currency format, you can find it under 'Funding Details' section\",\n          \"type\": \"string\"\n        },\n        \"AggregateStopLoss\": {\n          \"title\": \"Aggregate Stop Loss\",\n          \"description\": \"Must be in percentage format, example: 52%\",\n          \"type\": \"string\"\n        },\n        \"FundingArrangement\": {\n          \"title\": \"Funding Arrangement\",\n          \"description\": \"Accurately map the value across 'Surplus Share' returned to the employer **Example** if the value is  '1/3 retained by Cigna Healthcare, 2/3 returned to the Employer' then pick 2/3 and convert that to percentage which in this case is 67\\r\\n% a\\r\\nnd  'if you see 1/3 then mark it as 33%' , 'if you see 2/3 mark it as 67%', Must be in integer percentage format\",\n          \"type\": \"string\"\n        }\n      },\n      \"required\": [\n        \"IndividualStopLoss\",\n        \"AggregateStopLoss\",\n        \"FundingArrangement\"\n      ]\n    },\n    \"OutNetworkDetails\": {\n      \"title\": \"OutNetworkDetails\",\n      \"type\": \"object\",\n      \"properties\": {\n        \"DeductibleIndividual\": {\n          \"title\": \"Deductible Individual\",\n          \"type\": \"string\"\n        },\n        \"DeductibleFamily\": {\n          \"title\": \"Deductible Family\",\n          \"type\": \"string\"\n        },\n        \"Coinsurance\": {\n          \"title\": \"Coinsurance\",\n          \"type\": \"string\"\n        },\n        \"OutofPocketIndividual\": {\n          \"title\": \"Out of Pocket Individual\",\n          \"type\": \"string\"\n        },\n        \"OutofPocketFamily\": {\n          \"title\": \"Out of Pocket Family\",\n          \"type\": \"string\"\n        }\n      },\n      \"required\": [\n        \"DeductibleIndividual\",\n        \"DeductibleFamily\",\n        \"Coinsurance\",\n        \"OutofPocketIndividual\",\n        \"OutofPocketFamily\"\n      ]\n    },\n    \"Plan\": {\n      \"title\": \"Plan\",\n      \"type\": \"object\",\n      \"properties\": {\n        \"plan_information\": {\n          \"$ref\": \"#/definitions/PlanInformation\"\n        },\n        \"Level_Funded_Information\": {\n          \"$ref\": \"#/definitions/LevelFundedInformation\"\n        },\n        \"In_Network\": {\n          \"$ref\": \"#/definitions/InNetworkDetails\"\n        },\n        \"out_network\": {\n          \"$ref\": \"#/definitions/OutNetworkDetails\"\n        },\n        \"Enrollment_Actual\": {\n          \"$ref\": \"#/definitions/EnrollmentActual\"\n        },\n        \"rates\": {\n          \"$ref\": \"#/definitions/Rates\"\n        }\n      },\n      \"required\": [\n        \"plan_information\",\n        \"Level_Funded_Information\",\n        \"In_Network\",\n        \"out_network\",\n        \"Enrollment_Actual\",\n        \"rates\"\n      ]\n    },\n    \"PlanInformation\": {\n      \"title\": \"PlanInformation\",\n      \"description\": \"Extract correct information based on the following context strictly.\",\n      \"type\": \"object\",\n      \"properties\": {\n        \"Carrier\": {\n          \"title\": \"Carrier\",\n          \"description\": \"Only Carrier name, example: 'Anthem' and not 'Anthem Insurance'\",\n          \"type\": \"string\"\n        },\n        \"PlanName\": {\n          \"title\": \"Plan Name\",\n          \"description\": \"Name of the health insurance plan? give preference to what is stated across 'Plan Name'\",\n          \"type\": \"string\"\n        },\n        \"FundingType\": {\n          \"title\": \"Funding Type\",\n          \"description\": \"Funding type?\",\n          \"type\": \"string\"\n        },\n        \"PlanType\": {\n          \"title\": \"Plan Type\",\n          \"description\": \"Accurately map the value found in this field to one of to specific categories: 'PPO' or 'HSA' or 'HSA Compatible' or 'HSA Non-Compatible'\",\n          \"type\": \"string\"\n        },\n        \"NetworkType\": {\n          \"title\": \"Network Type\",\n          \"description\": \"Accurately map the value found in this field to one of to specific categories: 'OAP' or 'HSA Compatible'\",\n          \"type\": \"string\"\n        }\n      },\n      \"required\": [\n        \"Carrier\",\n        \"PlanName\",\n        \"FundingType\",\n        \"PlanType\",\n        \"NetworkType\"\n      ]\n    },\n    \"Rates\": {\n      \"title\": \"Rates\",\n      \"type\": \"object\",\n      \"properties\": {\n        \"EmployeeOnly\": {\n          \"title\": \"Employee Only\",\n          \"description\": \"Enrolled employee rate\",\n          \"type\": \"string\"\n        },\n        \"EmployeeSpouse\": {\n          \"title\": \"Employee Spouse\",\n          \"description\": \"Enrolled employee spouse rate\",\n          \"type\": \"string\"\n        },\n        \"EmployeeChild\": {\n          \"title\": \"Employee Child\",\n          \"description\": \"Enrolled employee child rate\",\n          \"type\": \"string\"\n        },\n        \"EmployeeFamily\": {\n          \"title\": \"EmployeeFamily\",\n          \"description\": \"Enrolled employee family rate\",\n          \"type\": \"string\"\n        }\n      },\n      \"required\": [\n        \"EmployeeOnly\",\n        \"EmployeeSpouse\",\n        \"EmployeeChild\",\n        \"EmployeeFamily\"\n      ]\n    }\n  },\n  \"type\": \"object\",\n  \"properties\": {\n    \"plans\": {\n      \"title\": \"Plans\",\n      \"type\": \"array\",\n      \"items\": {\n        \"$ref\": \"#/definitions/Plan\"\n      }\n    }\n  },\n  \"required\": [\n    \"plans\"\n  ]\n}",""")

In [8]:

create_message(thread.id,"user","""
context: Use the above schema to extract the data from the below text and analyse it.and give a json output.
BENEFIT SUMMARY
Administered by - Cigna Health and Life Insurance Co.
For - Luna Gourmet Coffee & Tea Company, LLC
HSA OAP Plan
HSAIN
Effective - 01/01/2024
Selection of a Primary Care Provider - your plan may require or allow the designation of a primary care provider. You have the right to designate any primary care
provider who participates in the network and who is available to accept you or your family members. If your plan requires designation of a primary care provider,
Cigna may designate one for you until you make this designation. For information on how to select a primary care provider, and for a list of the participating primary
care providers, visit www.mycigna.com or contact customer service at the phone number listed on the back of your ID card. For children, you may designate a
pediatrician as the primary care provider.
Direct Access to Obstetricians and Gynecologists - You do not need prior authorization from the plan or from any other person (including a primary care provider)
in order to obtain access to obstetrical or gynecological care from a health care professional in our network who specializes in obstetrics or gynecology. The health
care professional, however, may be required to comply with certain procedures, including obtaining prior authorization for certain services, following a pre-approved
treatment plan, or procedures for making referrals. For a list of participating health care professionals who specialize in obstetrics or gynecology, visit
www.mycigna.com or contact customer service at the phone number listed on the back of your ID card.

Your coverage includes a health savings account that you can use to pay for eligible out-of-pocket expenses.

Plan Highlights In-Network
Lifetime Maximum Unlimited
Your Plan’s Deductibles, Out-of-Pockets and benefit level limits accumulate on a
Plan Year Accumulation
calendar year basis unless otherwise stated.
Plan Coinsurance Plan pays 80%
Individual - Employee Only: $3,000
Plan Deductible
Family Maximum: $6,000
 Plan deductible always applies before any benefit copay/deductible or coinsurance.
 Plan deductible does not apply to preventive services.
 All family members contribute towards the family deductible. An individual cannot have claims covered under the plan coinsurance until the total family
deductible has been satisfied.
 This plan includes a combined Medical/Pharmacy plan deductible.  
DRAFT
 In-Network Generic preventive drugs and products included in the Preventive Package will not be subject to deductible. This may apply to drugs for: Asthma,
Cholesterol Lowering, Depression, Diabetes (including diabetic supplies but excluding continuous glucose monitor supplies), Heart Disease and Stroke, High
Blood Pressure, Osteoporosis, Prenatal Vitamins.
Note: Services where plan deductible applies are noted with a caret (^).

01/01/2024
ASO
Health Savings Account Open Access Plus - HSAIN

Facets - 28215063 - V 29 - 11/17/23 10:51 AM ET 1 of 10 ©Cigna 2023
Plan Highlights In-Network
Individual - Employee Only: $6,600
Plan Out-of-Pocket Maximum
Individual - within a Family: $6,600
Family Maximum: $13,200
 Plan deductible contributes towards your out-of-pocket maximum.  
 All benefit copays/deductibles contribute towards your out-of-pocket maximum.
 Covered expenses that count towards your out-of-pocket maximum include customer paid coinsurance and charges for Mental Health and Substance Use
Disorder.
 After each eligible family member meets his or her individual out-of-pocket maximum, the plan will pay 100% of their covered expenses. Or, after the family
out-of-pocket maximum has been met, the plan will pay 100% of each eligible family member's covered expenses.
 This plan includes a combined Medical/Pharmacy out-of-pocket maximum.
Benefit In-Network
Note: Services where plan deductible applies are noted with a caret (^). Plan deductible always applies before benefit copays/deductibles.
Physician Services - Office Visits
Primary Care Physician (PCP) Services/Office Visit Plan pays 80% ^ 
Specialty Care Physician Services/Office Visit Plan pays 80% ^     
Surgery Performed in Physician's Office Covered same as Physician Services - Office Visit
Virtual Care
Dedicated Virtual Providers - MDLIVE
MDLIVE Urgent Virtual Care Services Plan pays 80% ^
MDLIVE Primary Care Services Plan pays 80% ^
MDLIVE Specialty Care Services Plan pays 80% ^
 Primary Care cost share applies to routine care. Virtual wellness screenings are payable under Preventive Care.
 For MDLIVE Behavioral Services, please refer to the Mental Health and Substance Use Disorder section (below).
 Lab services supporting a virtual visit must be obtained through dedicated labs.
 Includes charges for the delivery of medical and health-related services and consultations by dedicated virtual providers as medically appropriate through
audio, video, and secure internet-based technologies.
Virtual Physician Services - Office Visits
Primary Care Physician (PCP) Services/Office Visit Plan pays 80% ^ 
Specialty Care Physician Services/Office Visit Plan pays 80% ^     
 Physicians may deliver services virtually that are payable under other benefits (e.g., Preventive Care, Outpatient Therapy Services).
DRAFT
 Includes charges for the delivery of medical and health-related services and consultations as medically appropriate through audio, video, and secure internet-
based technologies that are similar to office visit services provided in a face-to-face setting.
Convenience Care Clinic
Convenience Care Clinic Plan pays 80% ^

Preventive Care
01/01/2024
ASO
Health Savings Account Open Access Plus - HSAIN

Facets - 28215063 - V 29 - 11/17/23 10:51 AM ET 2 of 10 ©Cigna 2023
Benefit In-Network
Note: Services where plan deductible applies are noted with a caret (^). Plan deductible always applies before benefit copays/deductibles.
Preventive Care Office Visit Plan pays 100%
Preventive Services Plan pays 100%
 Includes preventive Mammograms, Papanicolaou (Pap), Prostate Specific Antigen (PSA) tests and colorectal screenings.
 Diagnostic-related services are covered at the same level of benefits as other x-ray and lab services, based on place of service.   
Immunizations Plan pays 100%
Inpatient
Inpatient Hospital Facility Services Plan pays 80% ^
Note: Includes all Lab and Radiology services, including Advanced Radiological Imaging as well as Medical Specialty Drugs
Inpatient Hospital Physician's Visit/Consultation Plan pays 80% ^  
Inpatient Professional Services Plan pays 80% ^
 For services performed by Surgeons, Radiologists, Pathologists and Anesthesiologists
Outpatient
Outpatient Facility Services Plan pays 80% ^
Outpatient Professional Services Plan pays 80% ^
 For services performed by Surgeons, Radiologists, Pathologists and Anesthesiologists
Emergency Services
Emergency Room
Plan pays 80% ^
 Includes ER Physician Charges, Lab and Radiology including       
Advanced Radiological Imaging (ARI)
Urgent Care Facility
Plan pays 80% ^
 Includes Physician Charges, Lab and Radiology
Ambulance Plan pays 80% ^
Ambulance services used as non-emergency transportation (e.g., transportation from hospital back home) generally are not covered.     
Inpatient Services at Other Health Care Facilities
Skilled Nursing Facility, Rehabilitation Hospital, Sub-Acute Facilities
Plan pays 80% ^
 Annual Limit: 60 days
Laboratory Services
Physician’s Services/Office Visit Covered same as Physician Services - Office Visit
DRAFT
Independent Lab Plan pays 80% ^
Outpatient Facility Plan pays 80% ^
Radiology Services
Physician’s Services/Office Visit Covered same as Physician Services - Office Visit
Outpatient Facility Plan pays 80% ^

01/01/2024
ASO
Health Savings Account Open Access Plus - HSAIN

Facets - 28215063 - V 29 - 11/17/23 10:51 AM ET 3 of 10 ©Cigna 2023
Benefit In-Network
Note: Services where plan deductible applies are noted with a caret (^). Plan deductible always applies before benefit copays/deductibles.
Includes MRI, MRA, CAT Scan, PET Scan, etc.
Advanced Radiological Imaging (ARI)
Outpatient Facility Plan pays 80% ^
Physician’s Services/Office Visit Plan pays 80% ^
Outpatient Therapy Services
Outpatient Physical Therapy Plan pays 80% ^
Annual Limits:
 Physical Therapy – 20 visits
 Limits are not applicable to mental health conditions.

Note: Therapy visits, provided as part of an approved Home Health Care plan, accumulate to the applicable Home Health Care maximum.   
Outpatient Speech Therapy, Hearing Therapy and Occupational        
Plan pays 80% ^
Therapy
Annual Limits:
 Speech, Hearing and Occupational Therapies – 20 visits
 Limits are not applicable to mental health conditions for Speech and Occupational Therapies.

Note: Therapy visits, provided as part of an approved Home Health Care plan, accumulate to the applicable Home Health Care maximum.   
Chiropractic Care Plan pays 80% ^
Annual Limit:
 Chiropractic Care – 12 visits
Hospice
Inpatient Facilities Plan pays 80% ^
Outpatient Services Plan pays 80% ^
Note: Includes Bereavement counseling provided as part of a hospice program.
Medical Pharmaceutical Drugs
Cigna Pathwell Specialty℠ Network:
Plan pays 80% ^
Cigna Pathwell Specialty℠ Medical Pharmaceuticals
All other medical network or out-of-network providers:
DRAFT
Not Covered

Other Medical Pharmaceuticals Plan pays 80% ^

Note: This benefit only applies to the cost of Medical Pharmaceutical drugs administered. Related Facility, Office Visit or Professional charges are covered according
to the plan design.

01/01/2024
ASO
Health Savings Account Open Access Plus - HSAIN

Facets - 28215063 - V 29 - 11/17/23 10:51 AM ET 4 of 10 ©Cigna 2023
Benefit In-Network
Note: Services where plan deductible applies are noted with a caret (^). Plan deductible always applies before benefit copays/deductibles.
Family Planning
Women’s Services Plan pays 100%
Includes contraceptive devices as ordered or prescribed by a physician and surgical sterilization services, such as tubal ligation (excludes reversals)
Men’s Services Not Covered
Includes surgical sterilization services, such as vasectomy (excludes reversals)
Abortion
Abortion Services Coverage varies based on Place of Service        
Note: Non-elective procedures only
Infertility
Infertility Treatment
Note: Coverage will be provided for the treatment of an underlying medical condition up to the point an infertility condition is diagnosed. Services will be covered as
any other illness.
Outpatient Dialysis Services
Physician’s Services/Office Visit Covered same as Physician Services - Office Visit
Home Dialysis
Covered same as plan's Home Health Care benefit
Note: Dialysis visits will not accumulate to Home Health Care maximum
Outpatient Facility Services Covered same as plan's Outpatient Facility Services benefit
Outpatient Professional Services Covered same as plan’s Outpatient Professional Services benefit
Other Health Care Facilities/Services
Home Health Care Plan pays 80% ^
 Annual Limit: 40 visits (The limit is not applicable to mental health and substance use disorder conditions.)
Organ Transplants Covered same as Inpatient benefit
 Services paid at in-network level if performed at Cigna LifeSOURCE Transplant Network® Facilities.
 Travel Maximum - Cigna LifeSOURCE Transplant Network® Facility Only: After the plan deductible is met, $10,000 maximum per Transplant
Condition-Specific Care Plan pays 100% ^
 Must be enrolled in the Condition-Specific Care program for orthopedic treatment prior to surgery and receive care from a specifically designated provider in
order to qualify.
 Includes specific services for surgery, including Facility and Professional charges from admission through discharge. Some limitations may apply.
DRAFT
 Travel Maximum - After the deductible is met, $600 per procedure 
Durable Medical Equipment and External Prosthetic Appliances       
Plan pays 80% ^
 Annual Limit: Unlimited
Breast Feeding Equipment and Supplies
 Limited to the rental of one breast pump per birth as ordered or 
Plan pays 100%
prescribed by a physician
 Includes related supplies
01/01/2024
ASO
Health Savings Account Open Access Plus - HSAIN

Facets - 28215063 - V 29 - 11/17/23 10:51 AM ET 5 of 10 ©Cigna 2023
Note: Services where plan deductible applies are noted with a caret (^).
Mental Health and Substance Use Disorder
Inpatient Mental Health Plan pays 80% ^
Outpatient Mental Health – Physician’s Office Plan pays 80% ^      
Outpatient Mental Health - MDLIVE Behavioral Services Plan pays 80% ^
Outpatient Mental Health – All Other Services Plan pays 80% ^      
Inpatient Substance Use Disorder Plan pays 80% ^
Outpatient Substance Use Disorder – Physician’s Office Plan pays 80% ^
Outpatient Substance Use Disorder - MDLIVE Behavioral Services Plan pays 80% ^
Outpatient Substance Use Disorder – All Other Services Plan pays 80% ^
Annual Limits:
 Unlimited maximum
Notes:
 Inpatient includes Acute Inpatient and Residential Treatment.    
 Outpatient - Physician's Office and MDLIVE Behavioral Services - may include Individual, family and group therapy, psychotherapy, medication management,
etc.
 Outpatient - All Other Services - may include Partial Hospitalization, Intensive Outpatient Services, Applied Behavior Analysis (ABA Therapy), etc.

Important Note on Mental Health and Substance Use Disorder Coverage: Covered medical services listed above, which are received to diagnose or treat a
Mental Health or Substance Use Disorder condition will be payable according to this section titled “Mental Health and Substance Use Disorder.”
Pharmacy In-Network
Cost Share and Supply
Med Pharmacy Cost Share                     Once the medical deductible is met then the customer is responsible for the cost share    
 Retail – up to 90-day supply
Retail:
(except Specialty up to 30-day supply)
You pay 20% ^
 Home Delivery – up to 90-day supply
Your plan pays 80% ^
(except Specialty up to 30-day supply)
 If you receive a supply of 34 days or less at home delivery of a 
Home Delivery:
Specialty Prescription Drug, the Specialty home delivery cost      
You pay 20% ^
share will be adjusted to reflect a Retail (per 30-day supply) cost
Your plan pays 80% ^
share.

DRAFT
01/01/2024
ASO
Health Savings Account Open Access Plus - HSAIN

Facets - 28215063 - V 29 - 11/17/23 10:51 AM ET 6 of 10 ©Cigna 2023
Pharmacy In-Network
 Cigna 90 Now Walgreens: Retail drugs for a 30 day supply may be obtained In-Network at a wide range of pharmacies across the nation although
prescriptions for a 90 day supply (such as maintenance drugs) will be available at select network pharmacies. CVS will be considered Out-of-Network for a
90 day supply.
 This plan will not cover out-of-network pharmacy benefits.       
 Cigna 90 Now Program: You can choose to fill your medications in a 30- or 90-day supply. If you choose to fill a 30-day prescription, it can be filled at any
network retail pharmacy or network home delivery pharmacy. If you choose to fill a 90-day prescription, it must be filled at a 90-day network retail pharmacy
or network home delivery pharmacy to be covered by the plan.       
 Specialty medications are used to treat an underlying disease which is considered to be rare and chronic including, but not limited to, multiple sclerosis,
hepatitis C or rheumatoid arthritis. Specialty Drugs may include high cost medications as well as medications that may require special handling and close
supervision when being administered.
 You can elect brand or generic with no penalty (MAC C).
 Exclusive specialty home delivery: Specialty medications must be filled through home delivery; otherwise you pay the entire cost of the prescription upon your
first fill. Some exceptions may apply.
 Your pharmacy benefits share an annual deductible and out-of-pocket maximum with the medical/behavioral benefits. The applicable cost share for covered
drugs applies after the combined deductible has been met.
Preventive Drugs:
Federally required preventive drugs will not be subject to deductible and will be provided at no charge. In addition, In-Network Generic preventive drugs and products
included in the Preventive Package will not be subject to deductible. This may apply to drugs for:
Asthma, Cholesterol Lowering, Depression, Diabetes (including diabetic supplies but excluding continuous glucose monitor supplies), Heart Disease and
Stroke, High Blood Pressure, Osteoporosis, Prenatal Vitamins       
Drugs Covered
Prescription Drug List:
Your Cigna Advantage Prescription Drug List includes a full range of drugs including all those required under applicable health care laws. Some of the more
expensive drugs are excluded when there are less expensive alternatives. To check which drugs are included in your plan, please log on to myCigna.com.
Some highlights:
 Coverage includes Self Administered injectable drugs, but excludes infertility drugs.
 Contraceptive devices and drugs are covered with federally required products covered at 100%.

DRAFT
01/01/2024
ASO
Health Savings Account Open Access Plus - HSAIN

Facets - 28215063 - V 29 - 11/17/23 10:51 AM ET 7 of 10 ©Cigna 2023
Pharmacy Program Information
Pharmacy Clinical Management: Essential
Your plan features drug management programs and edits to ensure safe prescribing, and access to medications proven to be the most reliable and cost effective for
the medical condition, including:
 Prior authorization requirements
 Step Therapy on select classes of medications and drugs new to the market
 Quantity limits, including maximum daily dose edits, quantity over time edits, duration of therapy edits, and dose optimization edits
 Age edits, and refill-too-soon edits
 Plan exclusion edits
 Current users of Step Therapy medications will be allowed one 30-day fill during the first three months of coverage before Step Therapy program applies.
 Your plan includes Specialty Drug Management features, such as prior authorization and quantity limits, to ensure the safe prescribing and access to
specialty medications.
 For customers with complex conditions taking a specialty medication, we will offer Accredo Therapeutic Resource Centers (TRCs) to provide specialty
medication and condition counseling. For customers taking a specialty medication not dispensed by Accredo, Cigna experts will offer this important specialty
medication and condition counseling.
Patient Assurance Program
Your plan includes the Patient Assurance Program, which waives the deductible and reduces the amount you owe for certain medications used to treat chronic
conditions included in the program. Additionally:
 Any amount you pay for these medications only count toward meeting your out-of-pocket maximum.
 Any discount provided by a pharmaceutical manufacturer for these medications only count toward meeting your out-of-pocket maximum.  
Additional Information
Cigna Diabetes Prevention Program in collaboration with Omada      
Cigna Diabetes Prevention Program in collaboration with Omada is a program to help you avoid the onset of diabetes, as well as health risks that might lead to heart
disease or a stroke. The program is covered by your health plan at the preventive level, just like for your wellness visit. Program participants have access to a
professional virtual health coach, an online support group, interactive lessons, and a smart-technology scale. The program will help you make small changes in your
eating, activity, sleep, and stress to achieve healthy weight loss through a series of 16 weekly lessons and tools to help you maintain weight loss over time. You will
also be offered the opportunity to join a gym for a low monthly fee and no enrollment fee.
Out-of-Network Emergency Services Charges
1. Emergency Services are covered at the In-Network cost-sharing level as required by applicable state or federal law if services are received from a non-participating
(Out-of-Network) provider.
2. The allowable amount used to determine the Plan's benefit payment for covered Emergency Services rendered in an Out-of-Network Hospital, or by an Out-of-
Network provider in an In-Network Hospital, is the amount agreed to by the Out-of-Network provider and Cigna, or as required by applicable state or federal law.
DRAFT
The member is responsible for applicable In-Network cost-sharing amounts (any deductible, copay or coinsurance). The member is not responsible for any charges
that may be made in excess of the allowable amount. If the Out-of-Network provider bills you for an amount higher than the amount you owe as indicated on the
Explanation of Benefits (EOB), contact Cigna Customer Service at the phone number on your ID card.

01/01/2024
ASO
Health Savings Account Open Access Plus - HSAIN

Facets - 28215063 - V 29 - 11/17/23 10:51 AM ET 8 of 10 ©Cigna 2023
Additional Information
Medicare Coordination
In accordance with the Social Security Act of 1965, this plan will pay as the Secondary plan to Medicare Part A and B as follows:     
(a) a former Employee such as a retiree, a former Disabled Employee, a former Employee's Dependent, or an Employee's Domestic Partner who is also eligible for
Medicare and whose insurance is continued for any reason as provided in this plan (including COBRA continuation);
(b) an Employee, a former Employee, an Employee’s Dependent, or former Employee’s Dependent, who is eligible for Medicare due to End Stage Renal Disease
after that person has been eligible for Medicare for 30 months.    

When a person is eligible for Medicare A and B as described above, this plan will pay as the Secondary Plan to Medicare Part A and B regardless if the person is
actually enrolled in Medicare Part A and/or Part B and regardless if the person seeks care at a Medicare Provider or not for Medicare covered services.
One Guide
Available by phone or through myCigna mobile application. One Guide helps you navigate the health care system and make the most of your health benefits and
programs.
Out-of-Area Services
 Coverage for services rendered outside a network area
For all other services, plan pays 80%
 ER and Ambulance paid the same as network services
after the in-network deductible is met
 Preventive care services covered at 100% for Out-of-Area
 In-Network Deductible and Out-of-Pocket maximums apply
Complete Care Management
Pre-authorization is required on all inpatient admissions and selected outpatient procedures, diagnostic testing, and outpatient surgery. Network providers are
contractually obligated to perform pre-authorization on behalf of their customers.
Pre-Existing Condition Limitation (PCL) does not apply.
Well-Being Solution: Core Plus
 Health Assessment
 Device/App Integration
 Personalized online content and data-driven actions
 Social connections/challenges
Definitions
Coinsurance - After you've reached your deductible, you and your plan share some of your medical costs. The portion of covered expenses you are responsible for
is called Coinsurance.
Copay - A flat fee you pay for certain covered services such as doctor's visits or prescriptions.
Deductible - A flat dollar amount you must pay out of your own pocket before your plan begins to pay for covered services.
Out-of-Pocket Maximum - Specific limits for the total amount you will pay out of your own pocket before your plan coinsurance percentage no longer applies. Once
DRAFT
you meet these maximums, your plan then pays 100 percent of the "Maximum Reimbursable Charges" or negotiated fees for covered services.
Place of Service - Your plan pays based on where you receive services. For example, for hospital stays, your coverage is paid at the inpatient level.
Prescription Drug List - The list of prescription brand and generic drugs covered by your pharmacy plan.
Professional Services - Services performed by Surgeons, Assistant Surgeons, Hospital Based Physicians, Radiologists, Pathologists and Anesthesiologists
Transition of Care - Provides in-network health coverage to new customers when the customer's doctor is not part of the Cigna network and there are approved
clinical reasons why the customer should continue to see the same doctor.

01/01/2024
ASO
Health Savings Account Open Access Plus - HSAIN

Facets - 28215063 - V 29 - 11/17/23 10:51 AM ET 9 of 10 ©Cigna 2023
Exclusions
What's Not Covered (This Is Not All Inclusive; check your plan documents for a complete list)
 Services that aren't medically necessary
 Experimental or investigational treatments, except for routine patient care costs related to qualified clinical trials as described in your plan document
 Accidental injury that occurs while working for pay or profit    
 Sickness for which benefits are paid or payable under any workers' compensation or similar law
 Services provided by government health plans
 Cosmetic surgery, unless it corrects deformities resulting from illness, breast reconstruction surgery after a mastectomy, or congenital defects of a newborn
or adopted child or child placed for adoption
 Dental treatments and implants
 Custodial care
 Surgical procedures for the improvement of vision that can be corrected through the use of glasses or contact lenses
 Vision therapy or orthoptic treatment
 Hearing aids
 Reversal of sterilization procedures
 Nonprescription drugs or anti-obesity drugs
 Non-emergency services incurred outside the United States        
 Bariatric surgery
 Infertility services
 Treatment of TMJ disorders and craniofacial muscle disorders     

These are only the highlights
This summary outlines the highlights of your plan. For a complete list of both covered and not covered services, including benefits required by your state, see your
employer's insurance certificate, service agreement or summary plan description -- the official plan documents. If there are any differences between this summary
and the plan documents, the information in the plan documents takes precedence.

All Cigna products and services are provided exclusively by or through operating subsidiaries of Cigna Corporation, including Cigna Health and Life Insurance
Company, Connecticut General Life Insurance Company, Evernorth Care Solutions, Inc., Evernorth Behavioral Health, Inc., Cigna Health Management, Inc., and
HMO or service company subsidiaries of Cigna Health Corporation.   

EHB State: CO

DRAFT
01/01/2024
ASO
Health Savings Account Open Access Plus - HSAIN

Facets - 28215063 - V 29 - 11/17/23 10:51 AM ET 10 of 10 ©Cigna 2023
DISCRIMINATION IS AGAINST THE LAW
Medical coverage

a grievance by sending an email to ACAGrievance@Cigna.com or by    
Cigna complies with applicable Federal civil rights laws and       
writing to the following address:
does not discriminate on the basis of race, color, national        
Cigna
origin, age, disability, or sex. Cigna does not exclude people     
or treat them differently because of race, color, national
Nondiscrimination Complaint Coordinator
origin, age, disability, or sex.
PO Box 188016
Chattanooga, TN 37422
Cigna:
If you need assistance filing a written grievance, please call the 
• Provides free aids and services to people with
number on the back of your ID card or send an email to
disabilities to communicate effectively with us, such as:
ACAGrievance@Cigna.com. You can also file a civil rights
– Qualified sign language interpreters
complaint with the U.S. Department of Health and Human
– Written information in other formats (large print,
Services, Office for Civil Rights electronically through the Office
audio, accessible electronic formats, other formats)
for Civil Rights Complaint Portal, available at
• Provides free language services to people whose
https://ocrportal.hhs.gov/ocr/portal/lobby.jsf, or by mail or phone at:
primary language is not English, such as:
U.S. Department of Health and Human Services
– Qualified interpreters
200 Independence Avenue, SW
– Information written in other languages
Room 509F, HHH Building
Washington, DC 20201
If you need these services, contact customer service at
1.800.368.1019, 800.537.7697 (TDD)
the toll-free number shown on your ID card, and ask a
Customer Service Associate for assistance.
Complaint forms are available at
http://www.hhs.gov/ocr/office/file/index.html.
If you believe that Cigna has failed to provide these services     
or discriminated in another way on the basis of race, color,       
national origin, age, disability, or sex, you can file
DRAFT
All Cigna products and services are provided exclusively by or through operating subsidiaries of Cigna Corporation, including Cigna Health and Life
Insurance Company, Connecticut General Life Insurance Company, Evernorth Care Solutions, Inc., Evernorth Behavioral Health, Inc., Cigna Health
Management, Inc., and HMO or service company subsidiaries of Cigna Health Corporation and Cigna Dental Health, Inc. The Cigna name, logos, and
other Cigna marks are owned by Cigna Intellectual Property, Inc. ATTENTION: If you speak languages other than English, language assistance
services, free of charge are available to you. For
current Cigna customers, call the number on the back of your ID card. Otherwise, call 1.800.244.6224
(TTY: Dial 711). ATENCIÓN: Si usted habla un idioma
que no sea inglés, tiene a su disposición servicios gratuitos de asistencia lingüística. Si es un cliente
actual de Cigna, llame al número que figura en el
reverso de su tarjeta de identificación. Si no lo es, llame al 1.800.244.6224 (los usuarios de TTY
deben llamar al 711).
 © 2021 Cigna.
896375b 05/21
DRAFT

""")
    

In [9]:
create_message(thread.id,"user","""
When extracting, follow the context and example given here:
 
context: important please note you should map the keys separated by slashes with the values separated by slashes
 
example:  
  Out-of-pocket maximum (individual/family)          $16,500/$33,000
  should be extracted as
  Out-of-pocket maximum individual : $16,500
  Out-of-pocket maximum family : $33,000
Instructions:
  - Include only the values; no explanations needed.
  - When you provide the JSON, make sure you provide it grouped by plan.
  - When extracting the plan name or product type, make sure it's complete like (KPLF 1500/15%/4000); do not provide short forms like (KPLF 1500).
  - Remember what you are extracting is a health insurance plan name; use your knowledge of how usually health insurance plans are represented based on that extract the plan names.
  - Only when there are more than one value category for rates, take the total maximum cost category as the value for rates; otherwise, pick the values as such.
  - If there are 3 plans, provide 3 plans as an array of JSON; similarly, if there are 4 plans, provide 4 plans as an array of JSON.
  - Please note don't restrict yourself to 3 or 4 entries; create as many as there are in the given text and provide the n plans as an array of JSON.
  - ONLY JSON IS ALLOWED as an answer. No explanation or other text is allowed. Do not wrap the answer in any other object. Just provide the answer as a valid JSON object.
  - '^' sign indicates after deductible. Strictly add 'After ded.' replacing '^' if you see it.
  - Example: '$75 + 90% ^' then cast it as '$75 + 90% After ded.'
  - Example: if there is only one plan and 'HSA Open Access Plus' is in the product name, consider it as 'HSA compatible' Plan Type; and only 'Open Access Plus' then it is Base Plan.
  - Example: if there are multiple plans and 'HSA Open Access Plus' is in the product name, consider it as 'HSA compatible' Plan Type; and only 'Open Access Plus' is in the product name, then it is buy-up Plan.
  - RX deductible is equal to In-Network Plan individual deductible.
```

""")

In [10]:
run_assistant(thread,assistant)

```json
{
  "plans": [
    {
      "plan_information": {
        "Carrier": "Cigna Health and Life Insurance Co.",
        "PlanName": "HSA Open Access Plus",
        "FundingType": "HSAIN",
        "PlanType": "HSA",
        "NetworkType": "OAP"
      },
      "Level_Funded_Information": {
        "IndividualStopLoss": "$6,600",
        "AggregateStopLoss": "NA",
        "FundingArrangement": "NA"
      },
      "In_Network": {
        "DeductibleIndividual": "$6,600",
        "DeductibleFamily": "$13,200",
        "OutofPocketFamily": "Family Maximum: $13,200",
        "OutofPocketIndividual": "Individual - within a Family: $6,600",
        "OfficeVisit": "Plan pays 80% After ded.",
        "Coinsurance": "NA",
        "InpatientHospitalServices": "Plan pays 80% After ded.",
        "OutpatientSurgery": "Plan pays 80% After ded.",
        "SpecialtyOfficeVisit": "Plan pays 80% After ded.",
        "UrgentCare": "Plan pays 80% After ded.",
        "EmergencyRoom": "Plan pays 80% After

In [11]:
create_message(thread.id,"user","""
context: Use the above schema to extract the data from the below text and analyse it.and give a json output.
Blue Hill at Stone Barns LLC
Effective Date: 01-01-2024
® ®
Open AccessElect Choice- New York

PLAN DESIGN & BENEFITS
PROVIDED BY AETNA LIFE INSURANCE COMPANY

PLAN FEATURES IN-NETWORK
Benefit limitations - Some service or supplies have limits on them per year. There might be a maximum number of
visits or days, or a dollar limit per year. In such cases, the benefit year begins on January 1 (unless otherwise noted).
Refer to your plan documents to learn more.
Deductible (per calendar year) None Individual
None Family
Member coinsurance Covered 100%
Applies to all expenses except as noted.
Out-of-pocket limit (per calendar  $500 per Individual
year)
$1,000 per Family
Some of your cost sharing may not count toward the out-of-pocket limit.
Your pharmacy expenses count toward your out-of-pocket limit.      
In-network expenses include coinsurance/copays and deductibles.    
Your family will have one out-of-pocket limit. You will meet it when the expenses of several family members add up to
the family out-of-pocket limit. No one person will have to pay more than the individual out-of-pocket limit amount.
Lifetime maximum
Unlimited except where otherwise indicated.
Primary care physician selection Encouraged
Referral requirement Not required
Telehealth consultations - You can access covered services for telehealth visits from different kinds of providers in
your plan. Log on to Aetna.com to see a list of telehealth providers. You'll also find more about your options, including
cost share amounts.
PREVENTIVE CARE IN-NETWORK
Routine adult physical exams/   Covered 100%
immunizations
1 exam every year
Routine well child         Covered 100%
exams/immunizations
• 7 exams in the first 12 months
• 3 exams from age 13 to 24 months
• 3 exams from age 25 to 36 months
• 1 exam every 12 months thereafter until age 22
Routine gynecological care exams Covered 100%
2 exams and pap smears per year, including related fees
Virtual primary care (VPC)     Covered 100%
preventive care consultations
Includes screening and counseling services for members age 18 and older
Routine mammogram Covered 100%
Recommended: One per year for members age 40 and over
Women's health Covered 100%
Includes: Screening for gestational diabetes, HPV (Human- Papillomavirus) DNA testing, counseling for sexually
transmitted infections, counseling and screening for human immunodeficiency virus, screening and counseling for
interpersonal and domestic violence, breastfeeding support, supplies and counseling.
Also includes: contraceptive methods (ACA mandated contraceptives, including contraceptives and devices you can't
get at a pharmacy), sterilization procedures (including tubal ligation), patient education and counseling. Limits may
apply.
Pre-natal maternity Covered 100%
Routine digital rectal exam Covered 100%
Recommended: For members age 40 and over

Page 1
Blue Hill at Stone Barns LLC
Effective Date: 01-01-2024
® ®
Open AccessElect Choice- New York

PLAN DESIGN & BENEFITS
PROVIDED BY AETNA LIFE INSURANCE COMPANY

Prostate-specific antigen test Covered 100%
Recommended: For members age 40 and over
Colorectal cancer screening Covered 100%
Recommended: For members age 45 and over
Routine eye exams $30 copay
1 routine exam per 24 months.
Routine hearing screening Covered 100%
PHYSICIAN SERVICES IN-NETWORK
Office visits to primary care    $20 office visit copay
physician (PCP)
Includes services of an internist, general physician, family practitioner or pediatrician.
Virtual primary care (VPC)     Covered 100%
consultations
Includes basic medical service consultations through a VPC vendor for members age 18 and older; refer to Aetna.com
for VPC vendor information
Telehealth consultation with non- $20 office visit copay
specialist
Specialist office visits $30 office visit copay
Telehealth consultation with    $30 office visit copay
specialist
Hearing exams Not Covered
Walk-in clinics $20 copay
Walk-in clinics are free-standing health care facilities. Sometimes they may be within a pharmacy, drug store,
supermarket, or other retail store. They offer some limited medical care and services.
Not walk-in clinics: Urgent care centers, emergency rooms, the outpatient department of a hospital, ambulatory
surgical centers, and physician offices.
Allergy testing Your cost sharing amount depends on the type of service and where you
receive it.
Allergy injections Your cost sharing amount depends on the type of service and where you
receive it. Covered 100% when an office visit charge is not applicable.
DIAGNOSTIC PROCEDURES IN-NETWORK
Diagnostic X-ray (Other than    Covered 100%
complex imaging services)
When your physician performs and bills for this service at their office, you pay your office visit cost share amount.
Diagnostic laboratory Covered 100%
When your physician performs and bills for this service at their office, you pay your office visit cost share amount.
Diagnostic complex imaging Covered 100%
When your physician performs and bills for this service at their office, you pay your office visit cost share amount.
EMERGENCY MEDICAL CARE IN-NETWORK
Urgent care provider $50 office visit copay
Non-urgent use of urgent care   Not Covered
provider
Emergency room $100 copay
Copay waived if admitted
Non-emergency care in an     Not Covered
emergency room
Emergency use of ambulance Covered 100%
Non-emergency use of ambulance Not Covered

Page 2
Blue Hill at Stone Barns LLC
Effective Date: 01-01-2024
® ®
Open AccessElect Choice- New York

PLAN DESIGN & BENEFITS
PROVIDED BY AETNA LIFE INSURANCE COMPANY

HOSPITAL CARE IN-NETWORK
Inpatient coverage $250 copay
Per confinement charge applied per admission, 2x limit per year, waived if readmitted to a hospital, regardless of
cause, within 90 days.
When you're admitted into a hospital for the care you need, your cost sharing amount counts toward all covered
benefits you receive.
Inpatient maternity coverage    $250 copay
(includes delivery and postpartum
care)
When you're admitted into a hospital for the care you need, your cost sharing amount counts toward all covered
benefits you receive.
Outpatient hospital Covered 100%
When you receive outpatient care at a hospital but don't stay overnight, your cost sharing amount counts toward all
covered benefits during your visit.
Outpatient surgery - hospital $50 copay
When you receive outpatient care at a hospital but don't stay overnight, your cost sharing amount counts toward all
covered benefits during your visit.
Outpatient surgery - freestanding  $50 copay
facility
When you receive outpatient care at a hospital but don't stay overnight, your cost sharing amount counts toward all
covered benefits during your visit.
MENTAL HEALTH SERVICES IN-NETWORK
Inpatient $250 copay
When you're admitted into a hospital for the care you need, your cost sharing amount counts toward all covered
benefits you receive.
Inpatient non-biologically based $250 copay
Your cost sharing applies to all covered benefits incurred during your inpatient stay.
Mental health office visits $20 copay
Crisis intervention services $20 copay
Mental health telehealth      $20 office visit copay
consultations
Other mental health services Covered 100%
When you receive outpatient care at a facility but don't stay overnight, your cost sharing amount counts toward all
covered benefits during your visit.
SUBSTANCE ABUSE IN-NETWORK
Inpatient $250 copay
When you're admitted into a hospital for the care you need, your cost sharing amount counts toward all covered
benefits you receive.
Residential treatment facility $250 copay
When you're admitted into a facility for the care you need, your cost sharing amount counts toward all covered benefits
you receive.
Substance abuse office visits $20 copay
Substance abuse telehealth    $20 office visit copay
consultations
Other substance abuse services Covered 100%
When you receive outpatient care at a facility but don't stay overnight, your cost sharing amount counts toward all
covered benefits during your visit.

Page 3
Blue Hill at Stone Barns LLC
Effective Date: 01-01-2024
® ®
Open AccessElect Choice- New York

PLAN DESIGN & BENEFITS
PROVIDED BY AETNA LIFE INSURANCE COMPANY

THERAPY SERVICES IN-NETWORK
Spinal manipulation therapy $30 copay
Outpatient short-term       $30 copay
rehabilitation
Limited to 50 visits per year
Includes physical, occupational, and speech therapies.
Habilitative physical therapy Covered 100%
Habilitative occupational therapy Covered 100%
Habilitative speech therapy Covered 100%
Autism related physical therapy Covered 100%
Autism related occupational    Covered 100%
therapy
Autism related speech therapy Covered 100%
Autism related behavioral therapy $20 copay
These benefits are combined with outpatient mental health visits   
Autism related applied behavior  Covered 100%
analysis
Covered same as any other Outpatient Mental Health benefit
OTHER SERVICES IN-NETWORK
Skilled nursing facility Covered 100%
Limited to 60 days per year
When you're admitted into a facility for the care you need, your cost sharing amount counts toward all covered benefits
you receive.
Home health care Covered 100%
Limited to 60 visits per year
Private duty nursing not included.
Limited to three visits per day by staff from a home health care agency. One visit equals a period of four hours or less.
Hospice care - inpatient Covered 100%
When you're admitted into a facility for the care you need, your cost sharing amount counts toward all covered benefits
you receive.
Hospice care - outpatient Covered 100%
When you receive outpatient care at a facility but don't stay overnight, your cost sharing amount counts toward all
covered benefits during your visit.
Private duty nursing Not Covered
Durable medical equipment Covered 100%
Diabetic supplies -- (if not covered  Covered same as any other medical expense.
under the prescription drug benefit)
You pay your prescription drug cost sharing amount if you have prescription
drug coverage. If not, you pay your PCP visit cost sharing amount. 
Infusion therapy - home/office $30 copay
Infusion therapy - outpatient    Your cost sharing amount depends on the type of service and where you
hospital/freestanding facility    receive it.
Gene-based, Cellular, and other  Your cost sharing amount depends on the type of service and where you
Innovative Therapies (GCIT™)   receive it.
$50 copay
In-network coverage is provided at GCIT™ designated facilities only.
Hearing aids Covered 100%
1 hearing aid per ear every 3 years
Transplants $250 copay
In-network coverage is only available at Institutes of Excellence (IOE)
contracted facility.
Page 4
Blue Hill at Stone Barns LLC
Effective Date: 01-01-2024
® ®
Open AccessElect Choice- New York

PLAN DESIGN & BENEFITS
PROVIDED BY AETNA LIFE INSURANCE COMPANY

Bariatric surgery $250 per admission copay
When you're admitted into a hospital for the care you need, your cost sharing amount counts toward all covered
benefits you receive.
Acupuncture $20 copay
Limited to 10 visits per year
FAMILY PLANNING IN-NETWORK
Infertility treatment Your cost sharing amount depends on the type of service and where you
receive it.
You have coverage for the diagnosis and treatment of the underlying cause of infertility.
Comprehensive infertility services Covered 100%
Artificial insemination and ovulation induction
Advanced Reproductive      Covered 100%
Technology (ART)
Limited to 3 courses of treatment per member's lifetime. Maximum applies to all procedures covered by any of our
plans except where prohibited by law. Coverage includes cryopreservation, storage and for iatrogenic only unlimited
storage and cryopreservation.
ART coverage includes: In vitro fertilization (IVF), zygote intrafallopian transfer (ZIFT), gamete intrafallopian transfer
(GIFT), cryopreserved embryo transfers, intracytoplasmic sperm injection (ICSI) or ovum microsurgery.
Vasectomy Covered 100%
Tubal ligation Covered 100%
PHARMACY IN-NETWORK
Pharmacy plan type Advanced Control Plan - Aetna
Prescription drug out-of-pocket  Prescription drug expenses apply to your medical out-of-pocket limit.
limit
Preferred generic drugs
Retail $10 copay
Mail order $30 copay
Preferred brand-name drugs
Retail $25 copay
Mail order $75 copay
Non-preferred generic and brand-name drugs
Retail $40 copay
Mail order $120 copay
Pharmacy day supply and requirements
Retail You can get up to a 30-day supply from Aetna National Network
Mail order You can get a 31-90-day supply from CVS Caremark® Mail Service
Pharmacy.
Specialty You can get up to a 30-day supply of specialty drugs     
You must fill all specialty drugs through our preferred specialty pharmacy
network.
Advanced Control Formulary Aetna Insured List

Page 5
Blue Hill at Stone Barns LLC
Effective Date: 01-01-2024
® ®
Open AccessElect Choice- New York

PLAN DESIGN & BENEFITS
PROVIDED BY AETNA LIFE INSURANCE COMPANY

Your prescription drug plan also includes:
• Diabetic supplies
• Insulin up to a $100 member payment maximum per fill per 30-day supply
• Sexual dysfunction drugs, including daily dose, additional 6 tablets a month for erectile dysfunction
• A limited list of over-the-counter medications when filled with a prescription
Family planning
• Oral and injectable fertility drugs included (physician charges for injections are not covered under RX, medical
coverage is limited).
• Contraceptives covered up to a 12-month supply. Contraceptive copay strategy applies.
The following are covered 100% in-network:
• Oral chemotherapy drugs
• Seasonal vaccinations
• Preventive vaccinations
• Affordable Care Act (ACA) eligible preventive medications and contraceptives
Refer to Aetna.com for a complete list of eligible prescription drugs.
Precertification requirements
Some covered prescription drugs need approval from us before we will cover the drug. If you are currently taking one
of these drugs when you switch to this plan, you may get one fill of your prescription within the first 90 days of starting
the plan.
Some covered prescription drugs require step therapy before we cover them. With step therapy, you must first try one
or more drugs before we will pay for drugs that require step therapy. If you are currently taking one of these drugs
when you switch to this plan. you may get one fill of your prescription within the first 90 days of starting this plan.
To get the most up-to-date precertification requirements and a list of drugs that require step therapy, see your plan
documents or go online to your member website.
Choose generics - Sometimes you or your provider may ask for a brand-name prescription drug when a generic is
available. If so, you will pay the brand-name copay plus the difference between the generic price and the brand-name
price.
GENERAL PROVISIONS
Dependents who are eligible to be Spouse, children from birth to age 26. Student status of children does not
on your plan           matter.

Plans are provided by: Aetna Health Inc. While this material is believed to be accurate as of the production date, it is
subject to change.

Health benefits and health insurance plans contain exclusions and limitations. Not all health services are covered.

See plan documents for a complete description of benefits, exclusions, limitations and conditions of coverage. Plan
features and availability may vary by location and are subject to change. Providers are independent contractors and
are not our agents. Provider participation may change without notice. We do not provide care or guarantee access to
health services.

If you are in a plan that requires the selection of a primary care physician and your primary care physician is part of an
integrated delivery system or physician group, your primary care physician will generally refer you to specialists and
hospitals that are affiliated with the delivery system or physician group.

Page 6
Blue Hill at Stone Barns LLC
Effective Date: 01-01-2024
® ®
Open AccessElect Choice- New York

PLAN DESIGN & BENEFITS
PROVIDED BY AETNA LIFE INSURANCE COMPANY

The following is a list of services and supplies that are generally not covered. However, your plan documents may
contain exceptions to this list based on state mandates or the plan design or rider(s) purchased by your employer.

• All medical and hospital services not specifically covered in, or which are limited or excluded by your plan
documents.
• Cosmetic surgery, including breast reduction.
• Custodial care.
• Dental care and dental X-rays.
• Donor egg retrieval
• Experimental and investigational procedures, except for coverage for medically necessary routine patient care costs
for members participating in a cancer clinical trial.
• Hearing aids
• Home births
• Immunizations for travel or work, except where medically necessary or indicated.
• Implantable drugs and certain injectable drugs including injectable infertility drugs.
• Infertility services, including artificial insemination and advanced reproductive technologies such as IVF, ZIFT, GIFT,
ICSI and other related services, unless specifically listed as covered in your plan documents.
• Long-term rehabilitation therapy.
• Non-medically necessary services or supplies.
• Outpatient prescription drugs (except for treatment of diabetes), unless covered by a prescription plan rider and over-
the-counter medications (except as provided in a hospital) and supplies.
• Radial keratotomy or related procedures.
• Reversal of sterilization.
• Services for the treatment of sexual dysfunction/enhancement, including therapy, supplies or counseling or
prescription drugs.
• Special duty nursing.
• Therapy or rehabilitation other than those listed as covered.    
• Weight control services including surgical procedures, medical treatments, weight control/loss programs, dietary
regimens and supplements, appetite suppressants and other medications; food or food supplements, exercise
programs, exercise or other equipment; and other services and supplies that are primarily intended to control weight or
treat obesity, including Morbid Obesity, or for the purpose of weight reduction, regardless of the existence of comorbid
conditions.

In case of emergency, call 911 or your local emergency hotline, or go directly to an emergency care facility.

Translation of this material into another language may be available. Please call Member Services at the number on
the back of your ID card.

Puede estar disponible la traduccion de este material en otro idioma. Por favor llame a Servicios al Miembro al
1-888-982-3862.

Plan features and availability may vary by location and group size.

For more information about Aetna plans, refer to www.aetna.com.    
Aetna and MinuteClinic, LLC (which either operates or provides certain management support services to MinuteClinic-
branded walk-in clinics) are both within the CVS Health family.    

© 2021 Aetna Inc.

Page 7

""")

In [12]:
create_message(thread.id,"user","""

When extracting make sure you follow the context and instructions given below::\\n\\n\r\n\r\n\r\ncontext = \"\"\"## Health Insurance Plan Data Extraction Guidelines\r\n### Context\r\n \r\nYou are tasked with extracting health insurance plan data from provided information, ensuring high accuracy and consistency in data representation. The data extraction requires attention to detail in mapping specific keys to their corresponding values based on the structure and examples provided below.\r\n \r\n### Instructions\r\n1. **Key-Value Mapping:**\r\n   - Extract key-value pairs by mapping keys separated by slashes with their corresponding values also separated by slashes.\r\n     - **Example:** \"Out-of-pocket maximum (individual/family) $16,500/$33,000\" should be extracted as:\r\n       - Out-of-pocket maximum individual: $16,500\r\n       - Out-of-pocket maximum family: $33,000\r\n \r\n2. **Plan Data Extraction:**\r\n   - Ensure all plans are extracted without omission.\r\n   - For `RX Tiers` and pharmacy-related data, values should be mapped across retails includes(Generic, Preferred Brand, Non-Preferred Brand, Specialty). Separate each item with a slash, Include only the values without explanations and without any prefix or suffix.\r\n     - **Example:** If you encounter multiple retail items like Retail Generic Copay = $w, Retail Pref Brand Copay = $x, Retail Non-Pref Brand Copay = $y, Retail Specialty Brand Copay extract as $w/$x/$y/$z.\r\n   - The \"Plan Type\" field should be one of 'PPO' or 'HSA' or 'HSA Compatible' or 'HSA Non-Compatible' based on quote provided.\r\n \r\n3. **Plan Name Extraction:**\r\n   - While picking plan name make sure to pick only what ever is mapped against 'plan' or 'plan name' \r\n     first see is there anything listed against 'plan name' extract that and assign\r\n     - **Example:**             HSA Open Access Plus\r\n                                HSA\r\n                Plan Offering   Quadruple Option\r\n                Plan Name       HSA (17276554)\r\n                Medical Management Model Included\r\n        in the above text even though first 3 rows seems like plan name since 'HSA (17276554)' is mapped against 'Plan Name' should pick 'HSA (17276554)' as value\r\n   - Extract the complete plan name or product type, avoiding abbreviations.\r\n     - **Example:** Use \"KPLF 1500/15%/4000\" instead of a shortened form like \"KPLF 1500\".\r\n   - Extract the plan name along with plan id/number\r\n     - **Example:** Use \"HSA (17276554)\" instead of HSA\r\n   - If the extraction is from SBC do include only the text after the keyword 'draft' in plan name\r\n     - **Example:** given \"Health Savings Account Open Access Plus - DRAFT HSA UW\" extract as \"HSA UW\"\r\n \r\n4. **Special Instructions:**\r\n   - Convert \"Level Funding\" to \"Level Funded\".\r\n   - Simplify carrier names, e.g., 'Cigna Healthcare' to 'Cigna'.\r\n   - For the Colorado market, the 'Outpatient Surgery' field should be left empty.\r\n   - Use the descriptions provided to determine the \"funding arrangement\" percentage.\r\n   - When extracting data for \"Enrolment\" make sure its extracted from \"Actual\" Column and should not take from \"Eligible\". Take a step by step approach like below\r\n     [step 1]: make sure the data extracted is revalant to enrollment\r\n     [step 2]: collect all the columns that has enrollment data (example: 'eligible' and 'actual')\r\n     [step 3]: now make sure you take data thats under 'actual' column and extract the data map it to the json schema\r\n     [step 4]: always note that data under 'eligible' column is irrelavant and not needed\r\n   - For 'Lab' field take the value from 'Lab Services - Ind. Lab'\r\n   - For 'X-ray' field take the value from 'Lab Services - Ind. Lab'\r\n   - For 'Advanced Imaging' field take the value from 'Adv. Radiology - Outpatient'\r\n   - When extracted value has '^' symbol make sure that is also part of the text extracted note it important not to truncate or trim it\r\n   - For 'Rx-Deductible' field if the current plan name has 'HSA' or 'HSA Compatible' assign the string 'Plan Deductible' else if the current plan name is 'PPO' or 'OAP' mark it as NA\r\n   - Do not include \"Plan Pays\" or 'plan pays' text. remove it if its present in any extracted value\r\n   - If you any of the field name has 'copay' text append 'copay' to the value extracted\r\n\r\n6. **Network Type Mapping:**\r\n   - The \"Network Type\" field should be either \"OAP\" or \"HSA OAP\" depending on the quote\r\n   - If 'HSA Open Access Plus' appears in the product name, consider it as 'HSA OAP'.\r\n   - If only 'Open Access Plus' or 'OAP' appears, map it as 'OAP'.\r\n \r\n7. **Plan Type Mapping:**\r\n   - The \"Plan Type\" field should be either 'HSA' or 'OAP' or 'PPO'\r\n   - If there is HSA in Plan Name  mark it as 'HSA' else its 'OAP' or 'PPO' depending on the plan name\r\n \r\n8. **Enrollment Mapping:**\r\n   - When extrating data for enrollment make sure what is extracted is a number\r\n   - When multiple columns has value for enrollment example: actual and eligible. pick values in actuals\r\n   - Always remember we need to map actuals to enrollment\r\n   - **Example:** Lets say we have the below text the values should be extracted from Actual column so the extracted values will be (11,0,1,2)                            \r\n                                      Eligible Actual\r\n                     Employee         71       11    \r\n                     Emp + Spouse     5        0     \r\n                     Emp + Child(ren) 4        1     \r\n                     Emp + Family     4        2\r\n\r\n9. **JSON Formatting:**\r\n   - Provide the extracted data in JSON format, grouped by plan.\r\n   - Ensure adherence to the conventions for accurate data representation.\r\n \r\n\"

""")

In [13]:
run_assistant(thread,assistant)

```json
{
  "plans": [
    {
      "plan_information": {
        "Carrier": "Aetna Life Insurance Company",
        "PlanName": "Open AccessElect Choice- New York",
        "FundingType": "NA",
        "PlanType": "OAP",
        "NetworkType": "OAP"
      },
      "Level_Funded_Information": {},
      "In_Network": {
        "DeductibleIndividual": "None",
        "DeductibleFamily": "None",
        "OutofPocketFamily": "No data available",
        "OutofPocketIndividual": "$500",
        "OfficeVisit": "$20 office visit copay",
        "Coinsurance": "Covered 100%",
        "InpatientHospitalServices": "Covered 100%",
        "OutpatientSurgery": "Covered 100%",
        "SpecialtyOfficeVisit": "Covered 100%",
        "UrgentCare": "$50 office visit copay",
        "EmergencyRoom": "$100 copay",
        "Lab": "Covered 100%",
        "XRay": "Covered 100%",
        "AdvancedImaging": "Covered 100%",
        "RXTiers": "NA",
        "RXDeductible": "NA"
      },
      "out_network": {
 

In [14]:
create_message(thread.id,"user","compare cigna and aetna outputs in json.")

In [15]:
run_assistant(thread,assistant)

```json
{
  "Cigna": {
    "plans": [
      {
        "plan_information": {
          "Carrier": "Cigna Health and Life Insurance Co.",
          "PlanName": "HSA OAP Plan",
          "FundingType": "NA",
          "PlanType": "HSA",
          "NetworkType": "HSA OAP"
        },
        "Level_Funded_Information": {},
        "In_Network": {
          "DeductibleIndividual": "None",
          "DeductibleFamily": "None",
          "OutofPocketFamily": "No data available",
          "OutofPocketIndividual": "$6,600",
          "OfficeVisit": "$20 office visit copay",
          "Coinsurance": "Plan pays 80%",
          "InpatientHospitalServices": "$250 copay",
          "OutpatientSurgery": "No data available",
          "SpecialtyOfficeVisit": "$30 office visit copay",
          "UrgentCare": "$50 office visit copay",
          "EmergencyRoom": "$100 copay",
          "Lab": "Covered 100%",
          "XRay": "Covered 100%",
          "AdvancedImaging": "Covered 100%",
          "RXTiers